# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
%load_ext sql
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# pd.read_sql用のコネクタ
conn = psycopg2.connect(**pgconfig)
# pd.to_sql用のcreate engine
engine = create_engine(dsl)
# MagicコマンドでSQLを書くための設定
%sql $dsl

df_customer = pd.read_sql(sql='select * from customer', con=conn)
df_category = pd.read_sql(sql='select * from category', con=conn)
df_product = pd.read_sql(sql='select * from product', con=conn)
df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
df_store = pd.read_sql(sql='select * from store', con=conn)
df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [2]:
df_receipt.head(5)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [3]:
a = df_receipt[["sales_ymd", "customer_id", "product_cd","amount"]]
a.head(5)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90


---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [4]:
a_new = a.rename(columns={"sales_ymd":"sales_date"})
a_new

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
...,...,...,...,...
104676,20180221,ZZ000000000000,P050101001,40
104677,20190911,ZZ000000000000,P071006005,218
104678,20170311,CS040513000195,P050405003,168
104679,20170331,CS002513000049,P060303001,148


---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [5]:
a[a["customer_id"] == 'CS018205000001'].head(5)

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102


---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [6]:
a.query("customer_id == 'CS018205000001' & amount >= 1000")

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [7]:
b = df_receipt[["sales_ymd", "customer_id", "product_cd", "quantity", "amount"]]
b.query("customer_id == 'CS018205000001' & amount >= 1000 | customer_id == 'CS018205000001' & quantity >= 5")

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [8]:
a.query("customer_id == 'CS018205000001' & amount >= 1000 & amount < 2000")

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [9]:
a = df_receipt[["sales_ymd", "customer_id", "product_cd","amount"]]
a.query('customer_id == "CS018205000001" & product_cd != "P071401019"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [10]:
df_store.query('(prefecture_cd != "13" & floor_area < 900)')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [11]:
df_store.query("store_cd.str.startswith('S14')", engine="python").head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [12]:
df_customer.query('customer_id.str.endswith("1")', engine="python").head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [13]:
df_store.query('address.str.contains("横浜市")', engine="python").head(5)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [14]:
df_customer.query("status_cd.str.contains('^[A-F]', regex=True)", engine="python").head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [15]:
df_customer.query("status_cd.str.contains('[1-9]$', regex=True)", engine="python").head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1


---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [16]:
df_customer.query("status_cd.str.contains('^[A-F].*[1-9]$', regex=True)", engine="python").head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [17]:
df_store.query("tel_no.str.contains('[0-9]{3}-[0-9]{3}-[0-9]{4}', regex=True)", engine="python").head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [18]:
df_customer.sort_values("birth_day", ascending=True).head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0


---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [19]:
df_customer.sort_values("birth_day", ascending=False).head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1


---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [20]:
df = pd.concat([df_receipt[["customer_id", "amount"]], df_receipt["amount"].rank(method='min', ascending=False)],axis=1)
df.columns = ["customer_id", "amount", "ranking"]
df.sort_values("ranking", ascending=True).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [21]:
df = pd.concat([df_receipt[["customer_id", "amount"]], df_receipt["amount"].rank(method='first', ascending=False)],axis=1)
df.columns = ["customer_id", "amount", "ranking"]
df.sort_values("ranking", ascending=True).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,5.0
72747,ZZ000000000000,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
596,CS015515000083,5280,9.0
11275,CS017414000114,5280,10.0


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [22]:
len(df_receipt)

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [23]:
df_receipt["customer_id"].nunique()

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [24]:
a = df_receipt[["store_cd", "amount", "quantity"]]

a.groupby("store_cd").sum().head()

,amount,quantity
store_cd,,
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403


---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [25]:
df_receipt.groupby("customer_id").max().head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,product_cd,quantity,amount
customer_id,,,,,,,,
CS001113000004,20190308,1552003200,S13001,112,2,P071401009,1,1200
CS001114000005,20190731,1564531200,S13001,112,2,P080803001,1,338
CS001115000010,20190405,1554422400,S13001,112,2,P090703035,1,2200
CS001205000004,20190625,1561420800,S13001,1192,2,P090206025,1,760
CS001205000006,20190224,1550966400,S13001,1192,2,P090202016,1,1100


---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [26]:
df_receipt.groupby("customer_id").min().head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,product_cd,quantity,amount
customer_id,,,,,,,,
CS001113000004,20190308,1552003200,S13001,112,1,P071001012,1,98
CS001114000005,20180503,1525305600,S13001,112,1,P050101001,1,40
CS001115000010,20171228,1514419200,S13001,112,1,P070101003,1,48
CS001205000004,20170914,1505347200,S13001,1132,1,P040501001,1,40
CS001205000006,20180207,1517961600,S13001,1152,1,P040101003,1,30


---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [27]:
df = df_receipt[["customer_id", "sales_ymd"]]
a = df.groupby("customer_id").max()
b = df.groupby("customer_id").min()

a.merge(b, how="left", on="customer_id").head()

,sales_ymd_x,sales_ymd_y
customer_id,,
CS001113000004,20190308,20190308
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207


---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [28]:
df = df_receipt[["store_cd", "amount"]].groupby("store_cd").mean().head()
df.sort_values("amount", ascending=False).head()

,amount
store_cd,
S12013,330.194130
S12029,315.623908
S12014,310.830261
S12007,307.688343
S12030,288.533727


---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [29]:
df = df_receipt[["store_cd", "amount"]].groupby("store_cd").median()
df.sort_values("amount", ascending=False).head()

,amount
store_cd,
S13052,190
S14010,188
S14050,185
S14040,180
S13003,180


---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [30]:
df = df_receipt[["store_cd", "product_cd"]].groupby("store_cd")
df.product_cd.apply(lambda x: x.mode()).reset_index().head()

,store_cd,level_1,product_cd
0,S12007,0,P060303001
1,S12013,0,P060303001
2,S12014,0,P060303001
3,S12029,0,P060303001
4,S12030,0,P060303001


---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [31]:
df = df_receipt[["store_cd","amount"]].groupby("store_cd").var().reset_index()
df.sort_values("amount", ascending=False).head()

,store_cd,amount
28,S13052,441863.252526
31,S14011,306442.242432
42,S14034,297068.392740
5,S13001,295558.842618
12,S13015,295427.197086


---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [32]:
std = df_receipt.groupby("store_cd").amount.std().reset_index()
std.sort_values("amount", ascending=False).head()

,store_cd,amount
28,S13052,664.727954
31,S14011,553.572256
42,S14034,545.039808
5,S13001,543.653237
12,S13015,543.532149


---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [33]:
df_receipt["amount"].quantile(q=np.arange(5)/4)

0.00       10.0
0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [34]:
df = df_receipt.groupby("store_cd").amount.mean().reset_index()
df[df["amount"] > 330].head()

,store_cd,amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [35]:
df = df_receipt.query('not customer_id.str.startswith("Z")',engine="python").groupby("customer_id").amount.sum().mean()
df

2547.742234529256

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [36]:
#顧客の売り上げ平均値を算出
customer_mean = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().mean()
print("全顧客の売り上げ平均:",customer_mean)

#顧客毎の売上金を算出
customer_amount = df_receipt.groupby('customer_id').amount.sum().reset_index()

#全体の平均値以上の顧客を算出
customer_amount[customer_amount["amount"] >= customer_mean].head()

全顧客の売り上げ平均: 2547.742234529256


,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639


---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [37]:
df_receipt.merge(df_store[["store_cd","store_name"]], how="inner", on="store_cd").head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店


---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [38]:
df_product.head(1)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,04,0401,040101,198.0,149.0


In [39]:
df_category.head(1)

,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
0,04,惣菜,0401,御飯類,040101,弁当類


In [40]:
df_product.merge(df_category[["category_small_cd", "category_small_name"]], how="inner", on="category_small_cd").head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,弁当類
1,P040101002,04,0401,040101,218.0,164.0,弁当類
2,P040101003,04,0401,040101,230.0,173.0,弁当類
3,P040101004,04,0401,040101,248.0,186.0,弁当類
4,P040101005,04,0401,040101,268.0,201.0,弁当類


---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [41]:
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
print(df_amount_sum.shape, df_amount_sum.head())

df_tmp = df_customer.query('gender_cd == "1" and not customer_id.str.startswith("Z")', engine='python')
print(df_tmp.shape, df_tmp.head())


df = pd.merge(df_tmp['customer_id'], df_amount_sum, how='left', on='customer_id').fillna(0)
print(df.shape)
df.head()

(8307, 2)       customer_id  amount
0  CS001113000004    1298
1  CS001114000005     626
2  CS001115000010    3044
3  CS001205000004    1988
4  CS001205000006    3337
(17918, 11)       customer_id customer_name gender_cd gender   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子         1     女性  1981-04-29   37  259-1113   
2  CS031415000172       宇多田 貴美子         1     女性  1976-10-04   42  151-0053   
3  CS028811000001        堀井 かおり         1     女性  1933-03-27   86  245-0016   
4  CS001215000145         田崎 美紀         1     女性  1995-03-29   24  144-0055   
6  CS015414000103         奥野 陽子         1     女性  1977-08-09   41  136-0073   

                  address application_store_cd application_date     status_cd  
0    神奈川県伊勢原市粟窪**********               S14021         20150905  0-00000000-0  
2     東京都渋谷区代々木**********               S13031         20150529  D-20100325-C  
3  神奈川県横浜市泉区和泉町**********               S14028         20160115  0-00000000-0  
4     東京都大田区仲六郷**********  

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0


---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [42]:
df_receipt.head(3)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170


In [43]:
# 売り上げ日数の列を追加
df = df_receipt

# 売り上げ日数の多い顧客の上位20件
df_day = df[~df.duplicated(subset=['customer_id', 'sales_ymd'])]
df_day = df_day.groupby("customer_id").sales_ymd.count().reset_index()
df_day = df_day.sort_values("sales_ymd", ascending=False).query("not customer_id.str.startswith('Z')", engine="python").head(20)

# 売り上げ金額の多い顧客の上位20件
df_maney = df.groupby("customer_id").amount.sum().reset_index()
df_maney = df_maney.sort_values("amount", ascending=False).query("not customer_id.str.startswith('Z')", engine="python").head(20)

# 上記２つのデータを外部結合
df_day.merge(df_maney, how="outer", on="customer_id").head()

,customer_id,sales_ymd,amount
0,CS040214000008,23.0,NaN
1,CS010214000010,22.0,18585.0
2,CS015415000185,22.0,20153.0
3,CS010214000002,21.0,NaN
4,CS028415000007,21.0,19127.0


---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [44]:
df_s = df_store.copy()
df_p = df_product.copy()

df_s["key"] = 0
df_p["key"] = 0

a = df_s.merge(df_p, how="outer", on="key")
len(a)

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [45]:
df = df_receipt[["sales_ymd", "amount"]].groupby("sales_ymd").amount.sum().reset_index()
df = pd.concat([df, df.shift()], axis=1)
df.columns = ["today", "money1", "yesterday", "money2"]
df["diff"] = df["money1"] - df["money2"]
df.head()

# 模範回答
df_sales_amount_by_date = df_receipt[['sales_ymd', 'amount']].groupby('sales_ymd').sum().reset_index()
df_sales_amount_by_date = pd.concat([df_sales_amount_by_date, df_sales_amount_by_date.shift()], axis=1)
df_sales_amount_by_date.columns = ['sales_ymd','amount','lag_ymd','lag_amount']
df_sales_amount_by_date['diff_amount'] = df_sales_amount_by_date['amount'] - df_sales_amount_by_date['lag_amount']
df_sales_amount_by_date.head(10)

,sales_ymd,amount,lag_ymd,lag_amount,diff_amount
0,20170101,33723,NaN,NaN,NaN
1,20170102,24165,20170101.0,33723.0,-9558.0
2,20170103,27503,20170102.0,24165.0,3338.0
3,20170104,36165,20170103.0,27503.0,8662.0
4,20170105,37830,20170104.0,36165.0,1665.0
5,20170106,32387,20170105.0,37830.0,-5443.0
6,20170107,23415,20170106.0,32387.0,-8972.0
7,20170108,24737,20170107.0,23415.0,1322.0
8,20170109,26718,20170108.0,24737.0,1981.0
9,20170110,20143,20170109.0,26718.0,-6575.0


---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [46]:
df_sales = df_receipt[["sales_ymd", "amount"]].groupby("sales_ymd").sum().reset_index()
for i in range(1,4):
    if i == 1:
        df_lag = pd.concat([df_sales, df_sales.shift(i)], axis=1)
    else:
        df_lag = pd.concat([df_lag, df_sales.shift(i)], axis=1)
df_lag.columns = ['sales_ymd', 'amount', 'lag_ymd_1', 'lag_amount_1', 'lag_ymd_2', 'lag_amount_2', 'lag_ymd_3', 'lag_amount_3']
df_lag.dropna().sort_values(["sales_ymd"]).head(10)

,sales_ymd,amount,lag_ymd_1,lag_amount_1,lag_ymd_2,lag_amount_2,lag_ymd_3,lag_amount_3
3,20170104,36165,20170103.0,27503.0,20170102.0,24165.0,20170101.0,33723.0
4,20170105,37830,20170104.0,36165.0,20170103.0,27503.0,20170102.0,24165.0
5,20170106,32387,20170105.0,37830.0,20170104.0,36165.0,20170103.0,27503.0
6,20170107,23415,20170106.0,32387.0,20170105.0,37830.0,20170104.0,36165.0
7,20170108,24737,20170107.0,23415.0,20170106.0,32387.0,20170105.0,37830.0
8,20170109,26718,20170108.0,24737.0,20170107.0,23415.0,20170106.0,32387.0
9,20170110,20143,20170109.0,26718.0,20170108.0,24737.0,20170107.0,23415.0
10,20170111,24287,20170110.0,20143.0,20170109.0,26718.0,20170108.0,24737.0
11,20170112,23526,20170111.0,24287.0,20170110.0,20143.0,20170109.0,26718.0
12,20170113,28004,20170112.0,23526.0,20170111.0,24287.0,20170110.0,20143.0


---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [47]:
df = pd.merge(df_receipt, df_customer, how ='inner', on="customer_id")
df["era"] = pd.cut(df.age, range(0,100,10))

# ピボットテーブルを作成
df_sales_summary = pd.pivot_table(df, index='era', columns='gender_cd', values='amount', aggfunc='sum').reset_index()
df_sales_summary.columns = ['era', 'male', 'female', 'unknown']
df_sales_summary

,era,male,female,unknown
0,"(0, 10]",0,0,0
1,"(10, 20]",1591,154504,4317
2,"(20, 30]",76990,1386076,51786
3,"(30, 40]",178587,684901,44656
4,"(40, 50]",14786,9616998,492748
5,"(50, 60]",53574,6512239,336936
6,"(60, 70]",276237,867377,68725
7,"(70, 80]",9667,8000,198
8,"(80, 90]",46360,269183,5111


---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [48]:
df_sales_summary = df_sales_summary.set_index('era'). \
        stack().reset_index().replace({'female':'01',
                                        'male':'00',
                                        'unknown':'99'}).rename(columns={'level_1':'gender_cd', 0: 'amount'})
df_sales_summary

,era,gender_cd,amount
0,"(0, 10]",00,0
1,"(0, 10]",01,0
2,"(0, 10]",99,0
3,"(10, 20]",00,1591
4,"(10, 20]",01,154504
5,"(10, 20]",99,4317
6,"(20, 30]",00,76990
7,"(20, 30]",01,1386076
8,"(20, 30]",99,51786
9,"(30, 40]",00,178587


---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [49]:
pd.concat([df_customer['customer_id'],pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%m%d')],axis = 1).head(10)

,customer_id,birth_day
0,CS021313000114,19810429
1,CS037613000071,19520401
2,CS031415000172,19761004
3,CS028811000001,19330327
4,CS001215000145,19950329
5,CS020401000016,19740915
6,CS015414000103,19770809
7,CS029403000008,19730817
8,CS015804000004,19310502
9,CS033513000180,19620711


---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [50]:
pd.concat([df_customer['customer_id'],pd.to_datetime(df_customer['application_date'])],axis = 1).head(10)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [51]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_ymd'].astype('str'))],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [52]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit="s")],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [53]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit="s").dt.strftime("%Y")],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018
1,1132,2,2018
2,1102,1,2017
3,1132,1,2019
4,1102,2,2018
5,1112,1,2019
6,1102,2,2018
7,1102,1,2019
8,1112,2,2017
9,1102,1,2019


---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [54]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt["sales_epoch"], unit="s").dt.strftime('%m')],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
5,1112,1,06
6,1102,2,12
7,1102,1,09
8,1112,2,05
9,1102,1,10


---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [55]:
pd.concat([df_receipt[["receipt_no","receipt_sub_no"]],
           pd.to_datetime(df_receipt["sales_epoch"], unit="s").dt.strftime('%d')],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
5,1112,1,05
6,1102,2,05
7,1102,1,22
8,1112,2,04
9,1102,1,10


---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [56]:
df = df_receipt.query("not customer_id.str.startswith('Z')", engine="python").groupby("customer_id").amount.sum().reset_index()

# apply関数を使って関数を適用させる。
df['sales_flg'] = df['amount'].apply(lambda x: 1 if x > 2000 else 0)
df.head(10)

,customer_id,amount,sales_flg
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [57]:
df_tmp = df_customer[['customer_id', 'postal_cd']].copy()
df_tmp['postal_flg'] = df_tmp['postal_cd'].apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)

# 指定された軸上で1つ以上の操作を使用して集計します。
df_tmp.merge(df_receipt, how='inner', on='customer_id').groupby('postal_flg').agg({'customer_id':'nunique'})

,customer_id
postal_flg,
0,3906
1,4400


---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [58]:
pd.concat([df_customer[['customer_id', 'address']], df_customer['address'].str[0:3].map({'埼玉県':'11',
                                                                                         '千葉県':'12', 
                                                                                         '東京都':'13', 
                                                                                         '神奈川':'14'})], axis=1).head(10)

,customer_id,address,address
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [59]:
df_sum = df_receipt.groupby("customer_id").amount.sum().reset_index()
p25 = df_sum["amount"].quantile(0.25)
p50 = df_sum["amount"].quantile(0.5)
p75 = df_sum["amount"].quantile(0.75)

def pct(x):
    if x < p25:
        return 1
    elif p25 <= x < p50:
        return 2
    elif p50 <= x < p75:
        return 3
    elif p75 <= x:
        return 4

df_sum["pct"] = df_sum["amount"].apply(lambda x: pct(x))
df_sum.head()

,customer_id,amount,pct
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3


---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [60]:
'''模範解答'''
# コード例1
df_customer_era = pd.concat([df_customer[['customer_id', 'birth_day']],
                             df_customer['age'].apply(lambda x: min(math.floor(x / 10) * 10, 60))],
                             axis=1)

df_customer_era.head(10)

# # コード例2
# df_customer['age_group'] = pd.cut(df_customer['age'], bins=[0, 10, 20, 30, 40, 50, 60, np.inf], right=False)
# df_customer[['customer_id', 'birth_day', 'age_group']].head(10)

,customer_id,birth_day,age
0,CS021313000114,1981-04-29,30
1,CS037613000071,1952-04-01,60
2,CS031415000172,1976-10-04,40
3,CS028811000001,1933-03-27,60
4,CS001215000145,1995-03-29,20
5,CS020401000016,1974-09-15,40
6,CS015414000103,1977-08-09,40
7,CS029403000008,1973-08-17,40
8,CS015804000004,1931-05-02,60
9,CS033513000180,1962-07-11,50


---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [61]:
df_customer_era["age"] = df_customer["gender_cd"] + df_customer_era["age"].astype(str)
df_customer_era.head(10)

,customer_id,birth_day,age
0,CS021313000114,1981-04-29,130
1,CS037613000071,1952-04-01,960
2,CS031415000172,1976-10-04,140
3,CS028811000001,1933-03-27,160
4,CS001215000145,1995-03-29,120
5,CS020401000016,1974-09-15,040
6,CS015414000103,1977-08-09,140
7,CS029403000008,1973-08-17,040
8,CS015804000004,1931-05-02,060
9,CS033513000180,1962-07-11,150


---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [62]:
pd.get_dummies(df_customer[['customer_id', 'gender_cd']], columns=['gender_cd']).head(10)

,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0
5,CS020401000016,1,0,0
6,CS015414000103,0,1,0
7,CS029403000008,1,0,0
8,CS015804000004,1,0,0
9,CS033513000180,0,1,0


---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [63]:
df = df_receipt.query("not customer_id.str.startswith('Z')", engine="python").groupby("customer_id").amount.sum().reset_index()
df['amount_ss'] = preprocessing.scale(df['amount'])
df.head(10)

,customer_id,amount,amount_ss
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [64]:
np.array(dir(preprocessing))

array(['Binarizer', 'FunctionTransformer', 'KBinsDiscretizer',
       'KernelCenterer', 'LabelBinarizer', 'LabelEncoder', 'MaxAbsScaler',
       'MinMaxScaler', 'MultiLabelBinarizer', 'Normalizer',
       'OneHotEncoder', 'OrdinalEncoder', 'PolynomialFeatures',
       'PowerTransformer', 'QuantileTransformer', 'RobustScaler',
       'StandardScaler', '__all__', '__builtins__', '__cached__',
       '__doc__', '__file__', '__loader__', '__name__', '__package__',
       '__path__', '__spec__', '_csr_polynomial_expansion', '_data',
       '_discretization', '_encoders', '_function_transformer', '_label',
       'add_dummy_feature', 'binarize', 'label_binarize', 'maxabs_scale',
       'minmax_scale', 'normalize', 'power_transform',
       'quantile_transform', 'robust_scale', 'scale'], dtype='<U25')

In [65]:
df['amount_ss'] = preprocessing.minmax_scale(df['amount'])
df.head(10)

,customer_id,amount,amount_ss
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771


---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [66]:
df['10'] = np.log10(df['amount'])
df.head(10)

,customer_id,amount,amount_ss,10
0,CS001113000004,1298,0.053354,3.113275
1,CS001114000005,626,0.024157,2.796574
2,CS001115000010,3044,0.129214,3.483445
3,CS001205000004,1988,0.083333,3.298416
4,CS001205000006,3337,0.141945,3.523356
5,CS001211000025,456,0.016771,2.658965
6,CS001212000027,448,0.016423,2.651278
7,CS001212000031,296,0.009819,2.471292
8,CS001212000046,228,0.006865,2.357935
9,CS001212000070,456,0.016771,2.658965


---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [67]:
df['log'] = np.log(df['amount'])
df.head(10)

,customer_id,amount,amount_ss,10,log
0,CS001113000004,1298,0.053354,3.113275,7.168580
1,CS001114000005,626,0.024157,2.796574,6.439350
2,CS001115000010,3044,0.129214,3.483445,8.020928
3,CS001205000004,1988,0.083333,3.298416,7.594884
4,CS001205000006,3337,0.141945,3.523356,8.112827
5,CS001211000025,456,0.016771,2.658965,6.122493
6,CS001212000027,448,0.016423,2.651278,6.104793
7,CS001212000031,296,0.009819,2.471292,5.690359
8,CS001212000046,228,0.006865,2.357935,5.429346
9,CS001212000070,456,0.016771,2.658965,6.122493


---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [68]:
df["利益"] = df_product["unit_price"] - df_product["unit_cost"]
df

,customer_id,amount,amount_ss,10,log,利益
0,CS001113000004,1298,0.053354,3.113275,7.168580,49.0
1,CS001114000005,626,0.024157,2.796574,6.439350,54.0
2,CS001115000010,3044,0.129214,3.483445,8.020928,57.0
3,CS001205000004,1988,0.083333,3.298416,7.594884,62.0
4,CS001205000006,3337,0.141945,3.523356,8.112827,67.0
...,...,...,...,...,...,...
8301,CS051212000001,336,0.011557,2.526339,5.817111,74.0
8302,CS051513000004,551,0.020899,2.741152,6.311735,74.0
8303,CS051515000002,265,0.008472,2.423246,5.579730,74.0
8304,CS052212000002,192,0.005301,2.283301,5.257495,74.0


---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [69]:
df_tmp = df_product.copy()
df_tmp['unit_profit_rate'] = (df_tmp['unit_price'] - df_tmp['unit_cost']) / df_tmp['unit_price']
riritu = df_tmp['unit_profit_rate'].mean(skipna=True)
riritu

0.24911389885176904

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [84]:
df_tmp = df_product.copy()
df_tmp['new_price'] = np.floor(df_tmp['unit_cost'] / 0.7)
df_tmp['new_profit_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_profit_rate
0,P040101001,04,0401,040101,198.0,149.0,212.0,0.297170
1,P040101002,04,0401,040101,218.0,164.0,234.0,0.299145
2,P040101003,04,0401,040101,230.0,173.0,247.0,0.299595
3,P040101004,04,0401,040101,248.0,186.0,265.0,0.298113
4,P040101005,04,0401,040101,268.0,201.0,287.0,0.299652
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,362.0,0.298343
7,P040101008,04,0401,040101,420.0,315.0,450.0,0.300000
8,P040101009,04,0401,040101,498.0,374.0,534.0,0.299625
9,P040101010,04,0401,040101,580.0,435.0,621.0,0.299517


---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [90]:
df_tmp = df_product.copy()
df_tmp['new_price'] = np.round(df_tmp['unit_cost'] / 0.7)
df_tmp['new_profit_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_profit_rate
0,P040101001,04,0401,040101,198.0,149.0,213.0,0.300469
1,P040101002,04,0401,040101,218.0,164.0,234.0,0.299145
2,P040101003,04,0401,040101,230.0,173.0,247.0,0.299595
3,P040101004,04,0401,040101,248.0,186.0,266.0,0.300752
4,P040101005,04,0401,040101,268.0,201.0,287.0,0.299652
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,363.0,0.300275
7,P040101008,04,0401,040101,420.0,315.0,450.0,0.300000
8,P040101009,04,0401,040101,498.0,374.0,534.0,0.299625
9,P040101010,04,0401,040101,580.0,435.0,621.0,0.299517


---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [91]:
df_tmp = df_product.copy()
df_tmp['new_price'] = np.ceil(df_tmp['unit_cost'] / 0.7)
df_tmp['new_profit_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_profit_rate
0,P040101001,04,0401,040101,198.0,149.0,213.0,0.300469
1,P040101002,04,0401,040101,218.0,164.0,235.0,0.302128
2,P040101003,04,0401,040101,230.0,173.0,248.0,0.302419
3,P040101004,04,0401,040101,248.0,186.0,266.0,0.300752
4,P040101005,04,0401,040101,268.0,201.0,288.0,0.302083
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,363.0,0.300275
7,P040101008,04,0401,040101,420.0,315.0,451.0,0.301552
8,P040101009,04,0401,040101,498.0,374.0,535.0,0.300935
9,P040101010,04,0401,040101,580.0,435.0,622.0,0.300643


---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [93]:
df_tmp = df_product.copy()
df_tmp['new_price'] = np.floor(df_tmp["unit_price"] * 1.1)
df_tmp['new_profit_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_profit_rate
0,P040101001,04,0401,040101,198.0,149.0,217.0,0.313364
1,P040101002,04,0401,040101,218.0,164.0,239.0,0.313808
2,P040101003,04,0401,040101,230.0,173.0,253.0,0.316206
3,P040101004,04,0401,040101,248.0,186.0,272.0,0.316176
4,P040101005,04,0401,040101,268.0,201.0,294.0,0.316327
5,P040101006,04,0401,040101,298.0,224.0,327.0,0.314985
6,P040101007,04,0401,040101,338.0,254.0,371.0,0.315364
7,P040101008,04,0401,040101,420.0,315.0,462.0,0.318182
8,P040101009,04,0401,040101,498.0,374.0,547.0,0.316271
9,P040101010,04,0401,040101,580.0,435.0,638.0,0.318182


---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [100]:
df_temp = df_receipt.merge(df_product, how='left', on='product_cd').groupby(['customer_id', 'category_major_cd'])['amount'].sum().unstack()

df_temp = df_temp[df_temp['07'] > 0]
df_temp['sum'] = df_temp.sum(axis=1)
df_temp['07_rate'] = df_temp['07'] / df_temp['sum']
df_temp.head(10)

category_major_cd,04,05,06,07,08,09,sum,07_rate
customer_id,,,,,,,,
CS001113000004,NaN,NaN,NaN,1298.0,NaN,NaN,1298.0,1.000000
CS001114000005,NaN,40.0,NaN,486.0,100.0,NaN,626.0,0.776358
CS001115000010,NaN,NaN,NaN,2694.0,NaN,350.0,3044.0,0.885020
CS001205000004,100.0,128.0,286.0,346.0,368.0,760.0,1988.0,0.174044
CS001205000006,635.0,60.0,198.0,2004.0,80.0,360.0,3337.0,0.600539
CS001212000027,248.0,NaN,NaN,200.0,NaN,NaN,448.0,0.446429
CS001212000031,NaN,NaN,NaN,296.0,NaN,NaN,296.0,1.000000
CS001212000046,NaN,NaN,NaN,108.0,NaN,120.0,228.0,0.473684
CS001212000070,NaN,NaN,148.0,308.0,NaN,NaN,456.0,0.675439


---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [113]:
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
                 how='inner', on='customer_id')

df_tmp = df_tmp.drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
df_tmp['elapsed_date'] = df_tmp['sales_ymd'] - df_tmp['application_date']
df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,1371 days
1,CS006214000001,2017-05-09,2015-02-01,828 days
2,CS006214000001,2017-06-08,2015-02-01,858 days
4,CS006214000001,2018-10-28,2015-02-01,1365 days
7,CS006214000001,2019-09-08,2015-02-01,1680 days
8,CS006214000001,2018-01-31,2015-02-01,1095 days
9,CS006214000001,2017-07-05,2015-02-01,885 days
10,CS006214000001,2018-11-10,2015-02-01,1378 days
12,CS006214000001,2019-04-10,2015-02-01,1529 days
15,CS006214000001,2019-06-01,2015-02-01,1581 days


---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [128]:
df_tmp = df_receipt[["customer_id", "sales_ymd"]].merge(df_customer[["customer_id", "application_date"]], how="inner", on="customer_id")
df_tmp = df_tmp.drop_duplicates()

df_tmp["sales_ymd"] = pd.to_datetime(df_tmp["sales_ymd"].astype("str"))
df_tmp["application_date"] = pd.to_datetime(df_tmp["application_date"])
df_tmp['elapsed_date'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x:
                                                                         relativedelta(x[0], x[1]).years * 12 + relativedelta(x[0], x[1]).months, axis=1)
df_tmp

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,45
1,CS006214000001,2017-05-09,2015-02-01,27
2,CS006214000001,2017-06-08,2015-02-01,28
4,CS006214000001,2018-10-28,2015-02-01,44
7,CS006214000001,2019-09-08,2015-02-01,55
...,...,...,...,...
65672,CS004411000027,2017-06-01,2015-05-17,24
65674,CS040513000029,2018-06-07,2015-09-15,32
65676,CS004613000146,2017-12-02,2016-08-13,15
65678,CS002314000037,2018-04-21,2015-12-02,28


---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [129]:
df_tmp = df_receipt[["customer_id", "sales_ymd"]].merge(df_customer[["customer_id", "application_date"]], how="inner", on="customer_id")
df_tmp = df_tmp.drop_duplicates()

df_tmp["sales_ymd"] = pd.to_datetime(df_tmp["sales_ymd"].astype("str"))
df_tmp["application_date"] = pd.to_datetime(df_tmp["application_date"])
df_tmp['elapsed_date'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x:
                                                                         relativedelta(x[0], x[1]).years, axis=1)
df_tmp

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,3
1,CS006214000001,2017-05-09,2015-02-01,2
2,CS006214000001,2017-06-08,2015-02-01,2
4,CS006214000001,2018-10-28,2015-02-01,3
7,CS006214000001,2019-09-08,2015-02-01,4
...,...,...,...,...
65672,CS004411000027,2017-06-01,2015-05-17,2
65674,CS040513000029,2018-06-07,2015-09-15,2
65676,CS004613000146,2017-12-02,2016-08-13,1
65678,CS002314000037,2018-04-21,2015-12-02,2


---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [133]:
df_tmp = df_receipt[["customer_id", "sales_ymd"]].merge(df_customer[["customer_id", "application_date"]], how="inner", on="customer_id")
df_tmp = df_tmp.drop_duplicates()

df_tmp["sales_ymd"] = pd.to_datetime(df_tmp["sales_ymd"].astype("str"))
df_tmp["application_date"] = pd.to_datetime(df_tmp["application_date"])

# エポック秒による経過時間を計算(timestamp()メソッドでdatetimeオブジェクトをUNIX時間に変換)
# 注：タイムスタンプは、ナノ秒を含むunix時間（10 ** 9で除算）
df_tmp['elapsed_date'] = (df_tmp['sales_ymd'].astype(np.int64) / 10**9) - \
                         (df_tmp['application_date'].astype(np.int64) / 10 **9)

# 1０件表示する
df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,118454400.0
1,CS006214000001,2017-05-09,2015-02-01,71539200.0
2,CS006214000001,2017-06-08,2015-02-01,74131200.0
4,CS006214000001,2018-10-28,2015-02-01,117936000.0
7,CS006214000001,2019-09-08,2015-02-01,145152000.0
8,CS006214000001,2018-01-31,2015-02-01,94608000.0
9,CS006214000001,2017-07-05,2015-02-01,76464000.0
10,CS006214000001,2018-11-10,2015-02-01,119059200.0
12,CS006214000001,2019-04-10,2015-02-01,132105600.0
15,CS006214000001,2019-06-01,2015-02-01,136598400.0


---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [134]:
df_tmp = df_receipt[['customer_id', 'sales_ymd']]
df_tmp = df_tmp.drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))

# 月曜のデータを追加する。(relativedelta(days=x.weekday())月曜：０, 火曜：１・・・)
df_tmp['monday'] = df_tmp['sales_ymd'].apply(lambda x: x - relativedelta(days=x.weekday()))

# 売上日（sales_ymd）から月曜日を引いて経過日数を求める
df_tmp['elapsed_weekday'] = df_tmp['sales_ymd'] - df_tmp['monday']
df_tmp.head(10)

,customer_id,sales_ymd,monday,elapsed_weekday
0,CS006214000001,2018-11-03,2018-10-29,5 days
1,CS008415000097,2018-11-18,2018-11-12,6 days
2,CS028414000014,2017-07-12,2017-07-10,2 days
3,ZZ000000000000,2019-02-05,2019-02-04,1 days
4,CS025415000050,2018-08-21,2018-08-20,1 days
5,CS003515000195,2019-06-05,2019-06-03,2 days
6,CS024514000042,2018-12-05,2018-12-03,2 days
7,CS040415000178,2019-09-22,2019-09-16,6 days
8,ZZ000000000000,2017-05-04,2017-05-01,3 days
9,CS027514000015,2019-10-10,2019-10-07,3 days


---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [140]:
df_customer.sample(frac=0.01)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
21582,CS008614000049,野上 美智子,1,女性,1952-03-08,67,157-0066,東京都世田谷区成城**********,S13008,20150907,2-20100830-1
17546,CS027414000036,羽田 光,1,女性,1972-06-04,46,251-0016,神奈川県藤沢市弥勒寺**********,S14027,20151215,C-20100929-C
21063,CS025314000059,土居 璃子,1,女性,1983-09-06,35,242-0024,神奈川県大和市福田**********,S14025,20160223,0-00000000-0
12975,CS003813000072,末永 奈央,1,女性,1937-07-01,81,182-0022,東京都調布市国領町**********,S13003,20180112,0-00000000-0
20391,CS038515000022,平尾 瞳,1,女性,1963-06-30,55,279-0043,千葉県浦安市富士見**********,S13038,20150325,9-20091212-C
...,...,...,...,...,...,...,...,...,...,...,...
15031,CS025615000032,北田 希,1,女性,1950-01-28,69,242-0024,神奈川県大和市福田**********,S14025,20150923,5-20100705-5
15762,CS002512000050,入江 花,1,女性,1960-07-25,58,187-0042,東京都小平市仲町**********,S13002,20150326,0-00000000-0
19069,CS011601000013,宮下 光博,0,男性,1953-03-30,66,211-0013,神奈川県川崎市中原区上平間**********,S14011,20150418,0-00000000-0
15851,CS002512000186,伊東 信吾,9,不明,1964-02-18,55,187-0013,東京都小平市回田町**********,S13002,20161021,0-00000000-0


---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [158]:
df_customer.sample(frac=0.1).groupby('gender_cd').agg({'customer_id' : 'count'})

,customer_id
gender_cd,
0,306
1,1779
9,112


---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [164]:
df = df_receipt.query("not customer_id.str.startswith('Z')", engine="python").groupby("customer_id").amount.sum().reset_index()

# 正規化・標準化(平均０、標準偏差１)：preprocessing.scale()関数を使う。
df['amount_ss'] = preprocessing.scale(df['amount'])

# 外れ値を抽出する(平均(mean)の ± 3σ(std)以上離れたものを抽出する)
# queryメソッドで条件に合うものを抽出。3σ以上離れたは、絶対値(abs)を用いる
df.query('abs(amount_ss)>=3').head(10)

,customer_id,amount,amount_ss
332,CS001605000009,18925,6.019921
1755,CS006415000147,12723,3.740202
1817,CS006515000023,18372,5.816651
1833,CS006515000125,12575,3.685800
1841,CS006515000209,11373,3.243972
1870,CS007115000006,11528,3.300946
1941,CS007514000056,13293,3.949721
1943,CS007514000094,15735,4.847347
1951,CS007515000107,11188,3.175970
1997,CS007615000026,11959,3.459372


---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [165]:
df = df_receipt.query("not customer_id.str.startswith('Z')", engine="python").groupby("customer_id").amount.sum().reset_index()
df['amount_ss'] = preprocessing.scale(df['amount'])

# 第一四分位と第三四分位の差であるIQRを求める
pct75 = df['amount'].quantile(0.75)
pct25 = df['amount'].quantile(0.25)
iqr = pct75 - pct25

# 第一四分位数-1.5×IQR(amount_low), 第三四分位数+1.5×IQR(amount_hight)をそれぞれ求める
amount_low = pct25 - (iqr * 1.5)
amount_hight = pct75 + (iqr * 1.5)
# 外れ値を抽出する(@で変数名)
df.query('amount > @amount_hight | amount < @amount_low').head(10)

,customer_id,amount,amount_ss
98,CS001414000048,8584,2.218796
332,CS001605000009,18925,6.019921
549,CS002415000594,9568,2.580493
1180,CS004414000181,9584,2.586374
1558,CS005415000137,8734,2.273933
1733,CS006414000001,9156,2.429051
1736,CS006414000029,9179,2.437505
1752,CS006415000105,10042,2.754725
1755,CS006415000147,12723,3.740202
1757,CS006415000157,10648,2.977477


---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [166]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [179]:
print(len(df_product))

df_product_1 = df_product.dropna()

print(len(df_product_1))

10030
10023


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [182]:
df_product_2 = df_product.fillna({'unit_price':np.round(np.nanmean(df_product['unit_price'])), 
                                  'unit_cost':np.round(np.nanmean(df_product['unit_cost']))})

# isnull().sum()で欠損を確認する
df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [186]:
df_product_3 = df_product.fillna({"unit_price":np.round(np.nanmedian(df_product["unit_price"])),
                                  "unit_cost":np.round(np.nanmedian(df_product["unit_cost"]))})

df_product_3.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [195]:
df_tmp = df_product.groupby("category_small_cd").agg({"unit_price" : "median", "unit_cost" : "median"}).reset_index()
df_tmp.columns = ['category_small_cd', 'median_price', 'median_cost']

df_product_4 = df_product.merge(df_tmp, how="inner", on="category_small_cd")

df_product_4["unit_price"] = df_product_4["unit_price"].mask(df_product_4["unit_price"].isnull(), df_product_4["median_price"].round())
df_product_4["unit_cost"] = df_product_4["unit_cost"].mask(df_product_4["unit_cost"].isnull(), df_product_4["median_cost"].round())

df_product_4.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
median_price          0
median_cost           0
dtype: int64

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [200]:
df_receipt.head(1)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158


In [199]:
df = df_customer[["customer_id", "application_date"]].copy()
re = df_receipt[["customer_id", "amount"]].copy()

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

# これで１００本終わりです。おつかれさまでした！